In [6]:
from diff_cap_packages import filters, stock_selection
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
%config Completer.use_jedi=False

In [15]:
start_date_backtesting = '2005-10-10'
end_date_backtesting = '2021-06-29'
step_backtesting = 1

dates = filters.prices.loc[start_date_backtesting:end_date_backtesting:step_backtesting].index.strftime("%Y-%m-%d").values
# remove = np.array(["2007-03-20"])
# dates = np.setdiff1d(dates, remove)
len(dates), dates

(3957,
 array(['2005-10-10', '2005-10-11', '2005-10-12', ..., '2021-06-25',
        '2021-06-28', '2021-06-29'], dtype=object))

In [19]:
# Trains and saves models
# Creates and saves predictions
# Display the process in a somewhat nice format

use_pycaret = False

num_days_back = 200
for index, end_date in enumerate(dates):
    
    suffix = "" if use_pycaret else "-npc"
    parent_path = os.path.join('models', end_date + "-linear-" + str(num_days_back) + suffix)

    print("--------------------")
    print("[", str(index + 1) + "/" + str(len(dates)), "]")
    print("Starting:", end_date + ".", "Using data from the previous", num_days_back, "days to train.")

    if not os.path.isfile(os.path.join(parent_path, 'completed.txt')):
        stock_selection.make_and_store_linear_models(end_date, num_days_back, force_overwrite=True, pycaret=use_pycaret)
    else:
        print("Model already trained and saved. Loading directly...")

    lrmodels = stock_selection.load_models(end_date, num_days_back, pycaret=use_pycaret)
    stock_selection.get_predictions(lrmodels, end_date, num_days_back, pycaret=use_pycaret)
    
    print("Finished.")
    print("--------------------")

NameError: name 'pycaret' is not defined

In [12]:
# Gets the predictions from the saved files.

all_predictions = dict.fromkeys(dates)

num_days_back = 200
for end_date in tqdm(dates):
    all_predictions[end_date] = stock_selection.get_predictions(None, end_date, num_days_back, pycaret=use_pycaret)

 26%|██████████▎                            | 1042/3957 [00:23<01:03, 45.71it/s]

Predictions not detected. Calculating and saving...



 26%|██████████▎                            | 1046/3957 [00:31<01:27, 33.46it/s]


KeyboardInterrupt: 

In [ ]:
all_returns = pd.DataFrame(index=dates)

for date in dates:
    predictions = all_predictions[date]
    buy_sell = stock_selection.get_buy_sell(predictions)
    all_returns.loc[date, "actual next day returns"] = stock_selection.get_actual_overall_pct_chg(buy_sell)

In [ ]:
all_returns["cumulative returns"] = [((np.array(all_returns.iloc[0 : idx+1, 0]) + 1).prod() - 1) for idx in range(0, all_returns.shape[0])]
all_returns

In [ ]:
end_date = "2020-03-13"
num_days_back = 200
pred = stock_selection.get_predictions(None, end_date, num_days_back, pycaret=use_pycaret)
buy_sell = stock_selection.get_buy_sell(pred)
analysis = stock_selection.get_buy_sell_analysis(buy_sell)
# pred
analysis
# How Percent Change is Calculated
# (analysis.iloc[0:5, 1].mean() - analysis.iloc[5:10, 1].mean())/2

In [ ]:
starting_amount = 100

end_amount = starting_amount
for date in dates:
    end_amount *= 1 + all_returns.loc[date, "actual next day returns"]

print("TRADING ON THE FOLLOWING DATES:\n", dates, "\n")
print("If starting with", starting_amount, "dollars,\nYou'd end up with", end_amount, "dollars")
print()
print("Overall percent change:", str(100 * (end_amount - starting_amount)/(starting_amount)) + "%")

In [ ]:
# Plot cumulative returns
import matplotlib.pyplot as plt
%matplotlib notebook

percent_returns = 100 * all_returns

percent_returns["cumulative returns"].plot(legend=True)

moving_average_periods = [5, 10, 20]
for moving_average_period in moving_average_periods:
    percent_returns[str(moving_average_period) + " day moving average"] = percent_returns["cumulative returns"].rolling(window=moving_average_period).mean()
    percent_returns[str(moving_average_period) + " day moving average"].plot(legend=True)

plt.xticks(rotation=45)
plt.tight_layout()